In [ ]:
savedmode_name="/model_seed2023_score0.984.pth"

In [ ]:
# %pip install --force-reinstall /kaggle/input/unidecode/Unidecode-1.3.8-py3-none-any.whl
%cp /kaggle/input/utility/* .

In [ ]:
# TODO:change path
from cfg import *
from util import *
from model1 import NLPModel
import gc

# Config.data_path='/kaggle/input/pii-detection-removal-from-educational-data/'
# Config.modelsavepath='/kaggle/input/deberbase0213'
tokenizer_path=Config.modelsavepath
modelpath=Config.modelsavepath+savedmode_name
modelpath

In [ ]:
Config.modelsavepath

In [ ]:
import json
from itertools import chain
from transformers import AutoModelForTokenClassification

In [ ]:
data = json.load(open(Config.data_path+"/train.json"))

In [ ]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})
ds

In [ ]:
tokenizer_path

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
idlabel= json.load(open(Config.modelsavepath+"//idlabel.json"))
id2label=idlabel['id2label']
label2id=idlabel['label2id']
id2label

In [ ]:
def inference_preprocesss(example,tokenizer,label2id):
    # rebuild text from tokens

    example['token_map']=[]
    example['berttokenids']=[]
    example['berttokenmask']=[]
    example['berttokentoken_type_ids']=[]
    example['offset_mapping']=[]

    tokens_split_list=[]
    trailing_whitespace_split_list=[]

    right_idx=0
    for i in range(0,len(example['tokens'])):
        if example['tokens'][i] == '\n\n':
            tokens_split_list.append(example['tokens'][right_idx:i+1])
            trailing_whitespace_split_list.append(example['trailing_whitespace'][right_idx:i+1])
            right_idx=i+1

    if  len(tokens_split_list)==0:
        tokens_split_list.append( example['tokens'])
        trailing_whitespace_split_list.append( example['trailing_whitespace'])

    idx=0
    for tokens_list,trailing_whitespace_list in zip(
        tokens_split_list,trailing_whitespace_split_list):

        text = []
        token_map = []

        for t,  ws in zip(
            tokens_list, trailing_whitespace_list
        ):
            text.append(t)
            token_map.extend([idx]*len(t))

            if ws:
                text.append(" ")
                token_map.append(-1)
            idx += 1
        # actual tokenization
        tokenized = tokenizer("".join(text), return_offsets_mapping=True)

        example['berttokenids'].append(tokenized['input_ids'])
        example['berttokenmask'].append(tokenized['attention_mask'])
        example['berttokentoken_type_ids'].append(tokenized['token_type_ids'])
        example['offset_mapping'].append(tokenized['offset_mapping'])
        example['token_map'].append(token_map)
        
    return example

In [ ]:
preprocesssed_ds=ds.map(inference_preprocesss, fn_kwargs={'tokenizer':tokenizer,'label2id':label2id},num_proc=10,desc="prepocessing data")

In [ ]:
# rebuid dataset 
tmp_pd=expanddataset(preprocesssed_ds,if_train=False)
print(tmp_pd['berttokenids'].str.len().agg(['mean','max','std','min']))
tmp_pd

In [ ]:
full_ds=Dataset.from_pandas(tmp_pd)
full_ds

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=NLPModel(id2label,label2id,Config.modelsavepath,training=False).to(device)
model.load_state_dict(torch.load(modelpath))

In [ ]:
data_collator = Collate(tokenizer=tokenizer,if_train=False)
val_dataloader=DataLoader(full_ds,batch_size=5,pin_memory=True,collate_fn=data_collator)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model.eval()

allprecitions=[]
for step,dataset in enumerate(tqdm(val_dataloader)):

    with torch.no_grad():
        ids = dataset["ids"].to(device,non_blocking=True)
        mask = dataset["mask"].to(device,non_blocking=True)
        tokentype = dataset["type_ids"].to(device,non_blocking=True)
        logit,loss = model(ids,mask,tokentype)
        allprecitions+=(logit2truepredic(logit,dataset['token_org_length']))
        del ids,mask,tokentype


In [ ]:
len(allprecitions)

In [ ]:
triplets = set()
document, token, label, token_str = [], [], [], []

for p,token_map, offsets,tokens, doc in zip(allprecitions,full_ds["token_map"], full_ds['offset_mapping'],full_ds["tokens"], full_ds["document"]):

    tmp_id=0
    assert(len(p)==len(offsets))
    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        
        label_pred = id2label[str(token_pred)]
        if start_idx + end_idx == 0: continue
        if token_map[start_idx] == -1:
            start_idx += 1

         # ignore "\n\n"
        while start_idx < len(token_map) and  tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map): break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred != "O" and token_id != -1:
            triplet = (label_pred, token_id, tokens[token_id])

            if triplet not in triplets:
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                triplets.add(triplet)
        tmp_id+=1

In [ ]:
df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label,
    "token_str": token_str
})
df["row_id"] = list(range(len(df)))
display(df.head(100))

In [ ]:
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)